# Bike Sharing Demand

Bike Sharing Demand는 주어진 자료들을 바탕으로 몇명이나 자전거를 빌릴 지를 예측하는 문제입니다.

## 필요한 모듈 호출

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('bmh')
%matplotlib inline

## 데이터 불러오기

In [ ]:
train = pd.read_csv("../input/bike-sharing-demand/train.csv")
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
print(test.shape)
test.head()

In [ ]:
test

## 컬럼 설명

- datatime - 시간, 연월일시분초 정보가 들어있는 컬럼.
- season - 계절 (봄, 여름, 가을, 겨울)
- hoilday - 공휴일 (0: 공휴일X, 1: 공휴일)
- workingday - 근무일 (0: 근무일X, 1:근무일)
- weather - 날씨 (1:맑음, 2:안개/구름, 3:약한 눈/비/천둥, 4:강한 눈/비/우박)
- temp - 온도
- atemp - 체감 온도
- humidity - 습도
- windspeed - 풍속
- casual - 비회원의 자전거 대여량
- registered - 회원의 자전거 대여량
- count - 총 자전거 대여량, casual + registered

## 데이터의 정보 및 결함 확인

In [ ]:
print('train 데이터의 정보')
train.info()

In [ ]:
test.info()

## 데이터 분석 (EDA)

## casual, registered, count

3개의 컬럼들은 모두 테스트 데이터엔 없는 컬럼입니다. count는 우리가 맞춰야할 컬럼이지만, 단순히 비회원과 회원이 빌린 자전거의 대수를 합한 값입니다.

따라서 이들의 특성을 정확히 반영하려면 casual과 registered에 대한 예측을 각각 한 다음 더하는 방법이 좋을 것 같습니다.

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(18, 5))

sns.distplot(train['casual'], ax=ax[0])
ax[0].set_title('casual distribution')

sns.distplot(train['registered'], ax=ax[1])
ax[1].set_title('registered distribution')

sns.distplot(train['count'], ax=ax[2])
ax[2].set_title('count distribution')

plt.show()

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(18, 5))

train['casual_log'] = np.log(train['casual'] + 1)
sns.distplot(train['casual_log'], ax=ax[0])
ax[0].set_title('casual_log distribution')

train['registered_log'] = np.log(train['registered'] + 1)
sns.distplot(train['registered_log'], ax=ax[1])
ax[1].set_title('registered_log distribution')

train['count_log'] = np.log(train['count'] + 1)
sns.distplot(train['count_log'], ax=ax[2])
ax[2].set_title('count_log distribution')

plt.show()

In [ ]:
print('casual이 0인 데이터의 개수')
train[train['casual'] == 0].shape

첫번째 시각화 자료를 통해 casual, registered, count의 분포를 살펴보았습니다.

모든 자료에서 대여한 자전거의 대수가 0대일 때가 가장 많았고, 100대 이하인 비중이 높습니다.

그러나 1000대에 가까운 자전거를 빌린 경우도 있어 분포가 많이 왜곡되어 보입니다.

두번째 시각화 자료를 통해 log를 씌운 casual, registered, count의 분포를 살펴보았습니다.

이번 경진대회의 평가 공식은 RESLE(Root Mean Squared Logarithmic Error)이기 때문에 log를 씌우고 1을 더한 값으로 분포를 살펴보았습니다.

casual_log는 자전거를 하나도 빌리지 않은 경우가 986개로 전체 데이터 10886개 중 약 10%로 많은 비중을 차지해 casual_log 값이 0인 분포가 많이 나타납니다.

이외에는 비교적 로그를 씌우지 않은 데이터보다 훨씬 정규분포에 가까운 형태를 보여줍니다.

**이를 통해 label로 casual과 registered를 사용할 때, log를 씌운 것이 좀 더 도움이 될 것이라고 추측할 수 있습니다.**

## datetime

In [ ]:
train["datetime"] = pd.to_datetime(train["datetime"], errors='coerce')
test['datetime'] = pd.to_datetime(test['datetime'], errors='coerce')

In [ ]:
train["datetime-year"] = train["datetime"].dt.year
train["datetime-month"] = train["datetime"].dt.month
train["datetime-day"] = train["datetime"].dt.day
train["datetime-hour"] = train["datetime"].dt.hour
train["datetime-minute"] = train["datetime"].dt.minute
train["datetime-second"] = train["datetime"].dt.second

train["datetime-dayofweek"] = train["datetime"].dt.dayofweek

train[["datetime", "datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-minute", "datetime-second", "datetime-dayofweek"]].head(3)

In [ ]:
test["datetime-year"] = test["datetime"].dt.year
test["datetime-month"] = test["datetime"].dt.month
test["datetime-day"] = test["datetime"].dt.day
test["datetime-hour"] = test["datetime"].dt.hour
test["datetime-minute"] = test["datetime"].dt.minute
test["datetime-second"] = test["datetime"].dt.second

test["datetime-dayofweek"] = test["datetime"].dt.dayofweek

test[["datetime", "datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-minute", "datetime-second", "datetime-dayofweek"]].head(3)

In [ ]:
train.loc[train['datetime-dayofweek'] == 0, 'datetime-dayofweek(str)'] = 'Mon'
train.loc[train['datetime-dayofweek'] == 1, 'datetime-dayofweek(str)'] = 'Tue'
train.loc[train['datetime-dayofweek'] == 2, 'datetime-dayofweek(str)'] = 'Wed'
train.loc[train['datetime-dayofweek'] == 3, 'datetime-dayofweek(str)'] = 'Thu'
train.loc[train['datetime-dayofweek'] == 4, 'datetime-dayofweek(str)'] = 'Fri'
train.loc[train['datetime-dayofweek'] == 5, 'datetime-dayofweek(str)'] = 'Sat'
train.loc[train['datetime-dayofweek'] == 6, 'datetime-dayofweek(str)'] = 'Sun'
train[['datetime', 'datetime-dayofweek', 'datetime-dayofweek(str)']]

In [ ]:
test.loc[test['datetime-dayofweek'] == 0, 'datetime-dayofweek(str)'] = 'Mon'
test.loc[test['datetime-dayofweek'] == 1, 'datetime-dayofweek(str)'] = 'Tue'
test.loc[test['datetime-dayofweek'] == 2, 'datetime-dayofweek(str)'] = 'Wed'
test.loc[test['datetime-dayofweek'] == 3, 'datetime-dayofweek(str)'] = 'Thu'
test.loc[test['datetime-dayofweek'] == 4, 'datetime-dayofweek(str)'] = 'Fri'
test.loc[test['datetime-dayofweek'] == 5, 'datetime-dayofweek(str)'] = 'Sat'
test.loc[test['datetime-dayofweek'] == 6, 'datetime-dayofweek(str)'] = 'Sun'
test[['datetime', 'datetime-dayofweek', 'datetime-dayofweek(str)']]

## datetime-all

In [ ]:
f, ax = plt.subplots(2, 3, figsize=(18, 10))

sns.barplot(data=train, x='datetime-year', y='count', ax=ax[0, 0])
ax[0, 0].set_title('Annual rental count')
sns.barplot(data=train, x='datetime-month', y='count', ax=ax[0, 1])
ax[0, 1].set_title('Monthly rental count')
sns.barplot(data=train, x='datetime-day', y='count', ax=ax[0, 2])
ax[0, 2].set_title('Daily rental count')
sns.barplot(data=train, x='datetime-hour', y='count', ax=ax[1, 0])
ax[1, 0].set_title('Rent count per hour')
sns.barplot(data=train, x='datetime-minute', y='count', ax=ax[1, 1])
ax[1, 1].set_title('Rent count per minute')
sns.barplot(data=train, x='datetime-second', y='count', ax=ax[1, 2])
ax[1, 2].set_title('Rent count per second')

plt.show()

**시각화 자료를 통해 연,월,일,시,분,초 별 count를 살펴보았습니다.
**
필요한 자료들을 더 자세히 살펴보는 것은 밑에서 이어서 하겠습니다. 여기선 거를 수 있는 데이터가 무엇인지 파악해보겠습니다.

- datetime-day : train 데이터에는 1일부터 19일까지만 존재합니다. 그 이후는 test 데이터에 존재합니다.
- datetime-minute : 모두 0이므로 의미가 없습니다.
- datetime-second : 모두 0이므로 의미가 없습니다.

**이를 통해 일,분,초의 정보는 feature에서 빼는 것이 좋다는 것을 알 수 있습니다.**

#### count by dayofweek

In [ ]:
plt.figure(figsize=(18, 5))
sns.barplot(data=train, x='datetime-dayofweek(str)', y='count').set_title('count by dayofweek')
plt.show()

요일별 대여량은 거의 비슷하다고 볼 수 있습니다.

#### casual vs registered

In [ ]:
train[['casual','registered', 'count']].sum()

비회원의 총 대여량은 약 390000대, 회원의 총 대여량을 1700000대로 회원의 총 대여량이 약 5배 정도입니다.

#### year

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(data=train, x='datetime-year', y='casual', ax=ax[0])
ax[0].set_title('Annual rental count: Non-member')
sns.barplot(data=train, x='datetime-year', y='registered', ax=ax[1])
ax[1].set_title('Annual rental count: Member')
sns.barplot(data=train, x='datetime-year', y='count', ax=ax[2])
ax[2].set_title('Annual rental count')

plt.show()

시각화 자료를 통해 year에 대한 casual, registered, count를 살펴보았습니다.

비회원, 회원 데이터 모두 2011년에 비해 2012년에 자전거 대여 횟수가 증가하였습니다.

이를 통해 자전거 대여 업체가 2011년 보다 2012년에 성장했다고 추측할 수 있습니다.

#### month

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(data=train, x='datetime-month', y='casual', ax=ax[0])
ax[0].set_title('Monthly rental count: Non-member')
sns.barplot(data=train, x='datetime-month', y='registered', ax=ax[1])
ax[1].set_title('Monthly rental count: Member')
sns.barplot(data=train, x='datetime-month', y='count', ax=ax[2])
ax[2].set_title('Monthly rental count')

plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 5))
sns.barplot(data=train, x='datetime-month', y='temp', ax=ax[0])
ax[0].set_title('Monthly temperature')
sns.barplot(data=train, x='datetime-month', y='atemp', ax=ax[1])
ax[1].set_title('Monthly sensible temperature')

plt.show()

첫번째 3개의 시각화 자료를 통해 month에 대한 casual, registered, count를 살펴보았습니다.
6-9월에 자전거 대여 수가 많고, 1-2월에 빌리는 사람이 적습니다. 이는 온도와 관련이 있을 것으로 추측할 수 있습니다.

두번째 2개의 시각화 자료를 통해 월별 온도와 체감온도를 살펴보았습니다.
위에서 나온 추측을 바탕으로 온도와 체감온도를 원별로 살펴본 결과 위의 그래프와 대부분 비슷하지만 1월과 12월의 온도차이는 크지 않은데 12월의 대여량이 많습니다.

월별 대여량은 온도와 관련이 있다고 볼 수 있습니다. 그러나 왜 12월의 대여량이 많은 지는 좀 더 살펴보아야 할 것 같습니다.


#### year-month

In [ ]:
train['year-month'] = train['datetime-year'].astype('str') + '-' + train['datetime-month'].astype('str')
train[['datetime', 'year-month']].head()

In [ ]:
plt.figure(figsize=(18, 5))
sns.barplot(data=train, x='year-month', y='count').set_title('Year-Month rent count')
plt.show()

year컬럼과 month 컬럼을 합쳐 연월별 자전거 대여량을 살펴보았습니다.

앞서 month 컬럼의 시간화 자료를 보면 1월과 12월은 겨울로 낮은 온도 또한 비슷하지만 대여량의 차이가 많이나는 것이 이상하다고 여겨졌습니다.
2011년 12월과 2012년 1월은 자전거 대여량의 차이가 크지 않습니다. 2011년 1월과 2012년 12월의 대여량 차이가 큰 이유는 
year 컬럼 시각화 자료에서 살펴보았듯 자전거 대여 업체가 성장하면서 이용자가 늘었다고 추측할 수 있습니다.

12월에 1월보다 많이 빌리는 것이 아닌 자전거 대여 업체의 성장에 따른 것으로 파악할 수 있습니다.

#### hour

In [ ]:
f, ax = plt.subplots(5, 1, figsize=(18, 25))
plt.subplots_adjust(hspace = 0.3)

sns.pointplot(data=train, x='datetime-hour', y='casual', ax=ax[0])
ax[0].set_title('Rent count per hour: Non-member')
sns.pointplot(data=train, x='datetime-hour', y='registered', ax=ax[1])
ax[1].set_title('Rent count per hour: Member')
sns.pointplot(data=train, x='datetime-hour', y='count', ax=ax[2])
ax[2].set_title('Rent count per hour')
sns.pointplot(data=train, x='datetime-hour', y='count', hue='workingday', ax=ax[3])
ax[3].set_title('Rent count per hour by workingday')
sns.pointplot(data=train, x='datetime-hour', y='count', hue='datetime-dayofweek(str)', ax=ax[4])
ax[4].set_title('Rent count per hour by dayofweek')

plt.show()

위쪽 3개 시각화 자료를 통해 시간별 casual, registered, count를 살펴보았습니다.

회원의 대여량인 registered는 count와 비슷한 형태입니다. 반면 비회원의 대여량인 casual은 완전히 다른 형태입니다.

casual과 register가 합쳐져서 count같은 형태를 띈다는 것은 비회원의 대여량 수보다 회원의 대여량 수가 훨씬 많다고 볼 수 있습니다.

4번째 시각화 자료를 통해 시간별 count를 근무일 유무로 나눠 살펴보았습니다.
근무를 하는 날엔 회원의 대여량과 비슷한 형태이고, 근무를 하지 않는 날엔 비회원의 대여량과 비슷한 형태입니다.

마지막 시각화 자료를 통해 시간별 count를 요일별로 나눠 살펴보았습니다.
보통 근무를 하는 날인 월,화,수,목,금은 회원의 대여량은 근무일의 대여량과 비슷하고,
휴일인 토,일은 비회원의 대여량, 쉬는 쉬는 날의 대여량과 비슷합니다.

시간별 대여량은 회원, 비회원 간의 차이가 존재합니다. 이는 근무일, 요일과 밀접한 관계가 있습니다. 회원은 근무일과 월~금, 비회원은 쉬는 날과 토,일에 자전거를 많이 빌린다고 추측할 수 있습니다.

### season, temp, atemp

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(data=train, x='season', y='count', ax=ax[0])
ax[0].set_title('Seasonal rent count')
sns.barplot(data=train, x='season', y='temp', ax=ax[1])
ax[1].set_title('Seasonal temperature')
sns.barplot(data=train, x='season', y='atemp', ax=ax[2])
ax[2].set_title('Seasonal sensible temperature')

plt.show()

In [ ]:
plt.figure(figsize=(18, 5))
sns.pointplot(data=train, x='datetime-month', y='count'
              ,hue='season').set_title('Monthly rent count: season')
plt.show()

**첫번째 3개의 시각화 자료를 통해 분기별 자전거 대여량과 분기별 온도, 체감온도를 살펴보았습니다.**

분기별 자전거 대여량은 3, 2, 4, 1분기 순서입니다.
분기별 대여량의 시각화 자료는 분기별 온도, 분기별 체감온도의 시각화 자료와 아주 유사합니다.
분기에 따른 대여량의 변화는 온도와 체감온도와 아주 밀접한 관계가 있다는 것을 알 수 있습니다.

**두번째 시각화 자료를 통해 월별 자전거 대여량을 분기로 나눠 살펴보았습니다.**
분기별로 확실히 대여량의 차이가 나기 때문에 학습할 때 의미있는 컬림이 될 수 있을 것 입니다.

**계절, 온도, 체감온도는 아주 밀접한 관계에 있으며 모두 대여량에 영향을 주는 요인입니다**

#### holiday

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 5))

sns.countplot(data=train, x='workingday', ax=ax[0])
ax[0].set_title('Amount of data of workingday')
sns.countplot(data=train, x='holiday', ax=ax[1])
ax[1].set_title('Amount of data of holiday')

plt.show()

총 10886개의 데이터 중 근무일의 데이터는 7412개 이고, 공휴일의 데이터는 311개 입니다.

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(18, 10))

sns.pointplot(data=train, x='datetime-hour', y='count', hue='holiday', ax=ax[0])
ax[0].set_title('Rent count per hour by holiday')
sns.pointplot(data=train, x='datetime-hour', y='count', hue='workingday', ax=ax[1])
ax[1].set_title('Rent count per hour by workingday')

**공휴일과 근무일에 따른 시간별 자전거 대여량을 살펴보았습니다.**

공휴일과 근무일은 정확히 반대 개념은 아닙니다. 하지만 시각화 자료를 보면 공휴일인 경우 비 근무일과 비슷한 형태를 보이고, 공휴일이 아닌 경우는 근무일과 비슷한 형태를 보여줍니다.

**따라서 공휴일 또한 자전거 대여량에 영향을 미치는 요소라 볼 수 있습니다.**

#### humidity, windspeed

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 5))

sns.distplot(train['humidity'], ax=ax[0])
ax[0].set_title('humdity distribution')
sns.distplot(train['windspeed'], ax=ax[1])
ax[1].set_title('windspeed distribution')

plt.show()

**좌측 시각화 자료를 통해 습도의 분포를 살펴보았습니다.**

습도는 비교적 눈에 띄는 데이터가 적고 고르게 퍼져 정규분포의 형태를 띄고 있습니다.

**우측 시각화 자료를 통해 풍속의 분포를 살펴보았습니다.**

풍속은 바람이 없는 날인 0을 제외하면 고른 분포를 보여줍니다.

**두 컬럼은 눈에 띄는 이상치가 있는 지를 파악하고 컬럼에 넣고, 빼면서 좋은 성능을 내는 지를 확인해 보는 것이 중요할 것입니다.**

#### weather

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 5))
sns.countplot(data=train, x='weather', ax=ax[0])
ax[0].set_title('Amount of data by weather')
sns.barplot(data=train, x='weather', y='count', ax=ax[1])
ax[1].set_title('Rent count by weather')
plt.show()

print('날씨가 1인 데이터 개수:', train[train['weather'] == 1].shape[0])
print('날씨가 2인 데이터 개수:', (train['weather'] == 2).sum())
print('날씨가 3인 데이터 개수:', (train['weather'] == 3).sum())
print('날씨가 4인 데이터 개수:', (train['weather'] == 4).sum())

좌측 시각화 자료를 날씨에 따른 데이터의 양을 살펴보았습니다.

1에 가까울수록 날씨가 좋고, 커질수록 날씨가 좋지 않은 것인데 1의 수가 가장 많습니다. 그러나 2,3의 데이터는 전체 데이터의 크기를 고려했을 때 무시할 정도는 아닙니다. 문제는 1개로 나타난 날씨 4입니다. 하지만 만개가 넘는 데이터 중 딱 1개의 데이터만 존재하기 때문에 큰 영향을 미칠 것 같진 않습니다.

우측 시각화 자료를 통해 날씨별 count를 살펴보았습니다.

날씨가 좋을수록 많이 빌리는 경향이 있습니다. 하지만 날씨가 매우 좋지 않은 날 또한 대여량이 많은데 이는 좌측에서 살펴본 것 처럼 딱 1개의 데이터만 가지고 있기 때문입니다.

## 데이터 전처리

#### season 전처리

In [ ]:
train_season = pd.get_dummies(train['season'], prefix='season')
train = pd.concat([train, train_season], axis=1)
train.head()

In [ ]:
train.describe()

In [ ]:
test_season = pd.get_dummies(test['season'], prefix='season')
test = pd.concat([test, test_season], axis=1)
test.head()

In [ ]:
test

#### weather 전처리

In [ ]:
train_weather = pd.get_dummies(train['weather'], prefix='weather')
train = pd.concat([train, train_weather], axis=1)
train.head()

In [ ]:
test_weather = pd.get_dummies(test['weather'], prefix='weather')
test = pd.concat([test, test_weather], axis=1)
test.head()

#### dayofweek 전처리

In [ ]:
train_dayofweek = pd.get_dummies(train['datetime-dayofweek(str)'], prefix='dayofweek')
train = pd.concat([train, train_dayofweek], axis=1)
train.head()

In [ ]:
test_dayofweek = pd.get_dummies(test['datetime-dayofweek(str)'], prefix='dayofweek')
test = pd.concat([test, test_dayofweek], axis=1)
test.head()

## 학습

#### feature 설정

In [ ]:
heatmap_data = train[['datetime-year', 'datetime-month', 'datetime-hour', 'weather', 'holiday', 'season', 'temp', 'atemp', 'humidity', 'windspeed']]

colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, fmt='.2f', vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

In [ ]:
feature_list = ['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'datetime-year', 'datetime-hour']
feature_list = feature_list + list(train_dayofweek.columns)
feature_list = feature_list + list(train_weather.columns)
feature_list = feature_list + list(train_season.columns)
feature_list

#### x_train, x_test, y_train_c, y_train_r 설정

- x_train: 학습할 feature 값
- x_test: 예측할 feature 값
- y_train_c: 학습할 label(비회원 대여량) 값
- y_train_r: 학습할 label(회원 대여량) 값

In [ ]:
x_train = train[feature_list]
x_train.head()

In [ ]:
x_test = test[feature_list]
x_test.head()

In [ ]:
test[feature_list]

In [ ]:
x_test

In [ ]:
y_train_c = train['casual_log']
y_train_c.head()

In [ ]:
y_train_r = train['registered_log']
y_train_r.head()

## Hyper Parameters 튜닝

#### Scorer 구현

In [ ]:
from sklearn.metrics import make_scorer
def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)
    
    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    return score

rmse_score = make_scorer(rmse)
rmse_score

#### csual_log에 대한 coarse search 

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score

# n_estimators = 300
# num_epoch = 100
# coarse_hyperparameters_list = []

# for epoch in range(num_epoch):
#     max_depth = np.random.randint(low=2, high=100)
#     max_features = np.random.uniform(low=0.1, high=1.0)
    
#     model = RandomForestRegressor(n_estimators=n_estimators,
#                                  max_depth=max_depth,
#                                  max_features=max_features,
#                                  n_jobs=-1,
#                                  random_state=37)
    
#     score = cross_val_score(model, x_train, y_train_c, cv=20, scoring=rmse_score).mean()
#     hyperparameters = {
#         'epoch': epoch,
#         'score': score,
#         'n_estimators': n_estimators,
#         'max_depth': max_depth,
#         'max_features': max_features,
#     }
    
#     coarse_hyperparameters_list.append(hyperparameters)
    
#     print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features: .6f}, Score = {score:.5f}")
          
# coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)
# coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by='score')
# print(coarse_hyperparameters_list.shape)
# coarse_hyperparameters_list.head()

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import cross_val_score

# n_estimators = 300
# num_epoch = 100
# coarse_hyperparameters_list = []

# for epoch in range(num_epoch):
#     max_depth = np.random.randint(low=56, high=94)
#     max_features = np.random.uniform(low=0.525012, high=0.565145)
    
#     model = RandomForestRegressor(n_estimators=n_estimators,
#                                  max_depth=max_depth,
#                                  max_features=max_features,
#                                  n_jobs=-1,
#                                  random_state=37)
    
#     score = cross_val_score(model, x_train, y_train_c, cv=20, scoring=rmse_score).mean()
#     hyperparameters = {
#         'epoch': epoch,
#         'score': score,
#         'n_estimators': n_estimators,
#         'max_depth': max_depth,
#         'max_features': max_features,
#     }
    
#     finer_hyperparameters_list.append(hyperparameters)
    
#     print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features: .6f}, Score = {score:.5f}")
          
# finer_hyperparameters_list = pd.DataFrame.from_dict(finer_hyperparameters_list)
# finer_hyperparameters_list = finer_hyperparameters_list.sort_values(by='score')
# print(finer_hyperparameters_list.shape)
# finer_hyperparameters_list.head()

#### 가장 좋은 파라매터를 통해 비회원 대여량 예측을 위한 모델 생성

In [ ]:
best_max_depth = 62.0
best_max_features = 0.547745
best_n_estimators = 3000

model = RandomForestRegressor(n_estimators=best_n_estimators,
                             max_depth=best_max_depth,
                             max_features=best_max_features,
                             random_state=37,
                             n_jobs=-1)
model

In [ ]:
model.fit(x_train, y_train_c)

#### 비회원 대여량 예측

In [ ]:
logC_predictions = model.predict(x_test)
print(logC_predictions.shape)
logC_predictions

In [ ]:
predictions_c = np.exp(logC_predictions) -1
print(predictions_c.shape)
predictions_c

#### 가장 좋은 파라매터를 통해 회원 대여량 예측을 위한 모델 생성

In [ ]:
best_max_depth = 63.0
best_max_features = 0.791589
best_n_estimators = 3000

In [ ]:
model = RandomForestRegressor(n_estimators=best_n_estimators,
                             max_depth=best_max_depth,
                             max_features=best_max_features,
                             random_state=37,
                             n_jobs=-1)
model

In [ ]:
model.fit(x_train, y_train_r)

#### 회원 대여량 예측

In [ ]:
logR_predictions = model.predict(x_test)
print(logR_predictions.shape)
logR_predictions

In [ ]:
predictions_r = np.exp(logR_predictions) - 1
print(predictions_r.shape)
predictions_r

## Submit

#### 비회원 대여량(casual) + 회원 대여량(registered) = 총 대여량 (count)

In [ ]:
predictions = predictions_c + predictions_r
predictions

In [ ]:
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
submission.head()

In [ ]:
submission.to_csv('bike_sharing_demand1.csv', index=False)